# Convolutional Neural Networks for Classifying Fashion-MNIST Dataset using Ignite
# Experiment tracking and visualization using Aim
This is a tutorial on using Ignite to train neural network models, setup experiments, validate models and then visualizing experiment data with Aim.

In this notebook, we will be doing classification of images using Convolutional Neural Networks 

We will be using the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) Fashion-MNIST is a set of 28x28 grayscale images of clothes.

![Fashion MNIST dataset](https://github.com/abdulelahsm/ignite/blob/update-tutorials/examples/notebooks/assets/fashion-mnist.png?raw=1)

Lets get started!

## Required Dependencies

We assume that `torch`, `ignite` and `aim` are already installed. We can install them using `pip`:

In [ ]:
!pip install pytorch-ignite

In [ ]:
!pip install aim

### Importing libraries

We import `torch`, `nn` and `functional` modules to create our models.

We also import `datasets` and `transforms` from torchvision for loading the dataset and applying transforms to the images in the dataset.

We import `Dataloader` for making train and validation loader for loading data into our model.

In [ ]:
import torch
import ignite
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

`Ignite` is a High-level library to help with training neural networks in PyTorch. It comes with an `Engine` to setup a training loop, various metrics, handlers and a helpful contrib section! 

Below we import the following:
* **Engine**: Runs a given process_function over each batch of a dataset, emitting events as it goes.
* **Events**: Allows users to attach functions to an `Engine` to fire functions at a specific event. Eg: `EPOCH_COMPLETED`, `ITERATION_STARTED`, etc.
* **Accuracy**: Metric to calculate accuracy over a dataset, for binary, multiclass, multilabel cases. 
* **Loss**: General metric that takes a loss function as a parameter, calculate loss over a dataset.
* **RunningAverage**: General metric to attach to Engine during training.
* **global_step_from_engine**: Helper method to setup global_step_transform function using another engine.
* **EarlyStopping**: Handler to stop training based on a score function.
* **ProgressBar**: Utility to easily track the progress of the training

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import global_step_from_engine, EarlyStopping
from ignite.contrib.handlers import ProgressBar

Then finally we import `aim`'s adapter designed for ignite to be able to track training metrics and h-params

In [ ]:
from aim.pytorch_ignite import AimLogger

The code below first sets up transform using `torhvision transfroms` for converting images to pytorch tensors and normalizing the images.

Next, we use `torchvision datasets` for dowloading the fashion mnist dataset and applying transforms which we defined above.

* `trainset` contains the training data.
* `validationset` contains the validation data

Next, we use `pytorch dataloader` for making dataloader from the train and validation sets.

In [ ]:
# transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = datasets.FashionMNIST('./data', download=True, train=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
validationset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform)
val_loader = DataLoader(validationset, batch_size=64, shuffle=True)

### CNN Model

Explanation of Model Architecture

* [Convolutional layers](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), the Convolutional layer is used to create a convolution kernel that is convolved with the layer input to produce a tensor of outputs.
* [Maxpooling layers](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html), the Maxpooling layer is used to downsample an input representation keeping the most active pixels from the previous layer.
* The usual [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) + [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html) layers to avoid overfitting and produce a 10-dim output.
* We had used [Relu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) Non Linearity for the model and [logsoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html) at the last layer because we are going to use the [NLLL loss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html).


In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.convlayer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.convlayer2 = nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(64*6*6,600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = x.view(-1,64*6*6)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return F.log_softmax(x,dim=1)

### Creating Model, Optimizer and Loss

Below we create an instance of the CNN model. The model is placed on a device and then a loss function of `negative log likelihood loss` and `Adam optimizer` with learning rate of 0.001 are setup. 

In [ ]:
# creating model,and defining optimizer and loss
model = CNN()
# moving model to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

### Training and Evaluating using Ignite

### Instantiating Training and Evaluating Engines

Below we create 3 engines, a trainer, an evaluator for the training set and an evaluator for the validation set, by using the `create_supervised_trainer` and `create_supervised_evaluator` and passing the required arguments.

We import the metrics from `ignite.metrics` which we want to calculate for the model. Like `Accuracy`, `ConfusionMatrix`, and `Loss` and we pass them to `evaluator` engines which will calculate these metrics for each iteration.

* `training_history`: it stores the training loss and accuracy
* `validation_history`:it stores the validation loss and accuracy
* `last_epoch`: it stores the last epoch untill the model is trained


In [ ]:
# defining the number of epochs
epochs = 12
# creating trainer,evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
metrics = {
    'accuracy':Accuracy(),
    'nll':Loss(criterion),
    'cm':ConfusionMatrix(num_classes=10)
}
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
training_history = {'accuracy':[],'loss':[]}
validation_history = {'accuracy':[],'loss':[]}
last_epoch = []

### Metrics - RunningAverage

To start, we will attach a metric of `RunningAverage` to track a running average of the scalar loss output for each batch. 

In [ ]:
RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

### EarlyStopping - Tracking Validation Loss

Now we will setup a `EarlyStopping` handler for this training process. EarlyStopping requires a score_function that allows the user to define whatever criteria to stop trainig. In this case, if the loss of the validation set does not decrease in 10 epochs, the training process will stop early. Since the `EarlyStopping` handler relies on the validation loss, it's attached to the `val_evaluator`. 

In [ ]:
def score_function(engine):
    val_loss = engine.state.metrics['nll']
    return -val_loss

handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, handler)

The below function will trigger training evaluation and validation after each epoch is completed.

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    train_evaluator.run(train_loader)
    val_evaluator.run(val_loader)


### Attaching Custom Functions to Engine at specific Events

Below you will see how to create and use `AimLogger`.

Create a logger

In [ ]:
aim_logger = AimLogger()

Log experiment parameters:

In [ ]:
aim_logger.log_params({
    "model": model.__class__.__name__,
    "pytorch_version": str(torch.__version__),
    "ignite_version": str(ignite.__version__),
})

Attach the logger to the trainer to log training loss at each iteration.

In [ ]:
aim_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="train",
    output_transform=lambda loss: {'loss': loss}
)

Attach the logger to the evaluator on the training dataset and log `NLL`, `Accuracy` metrics after each epoch.
We setup `global_step_transform=global_step_from_engine(trainer)` to take the epoch
of the `trainer` instead of `train_evaluator`.

In [ ]:
aim_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="train",
    metric_names=["nll", "accuracy"],
    global_step_transform=global_step_from_engine(trainer),
)

Attach the logger to the evaluator on the validation dataset and log `NLL`, `Accuracy` metrics after
each epoch.
We setup `global_step_transform=global_step_from_engine(trainer)` to take the epoch of the
`trainer` instead of `evaluator`.

In [ ]:
aim_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="val",
    metric_names=["nll", "accuracy"],
    global_step_transform=global_step_from_engine(trainer),
)

Attach the logger to the trainer to log optimizer's parameters, e.g. learning rate at each epoch iteration

In [ ]:
aim_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.EPOCH_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

### Run Engine

Finally, we'll attach a progress bar to the trainer via `loss` metric to track the training progress from console output
and then we will run the trainer for 12 epochs and monitor results.

In [ ]:
pbar = ProgressBar(persist=True, bar_format="")
pbar.attach(trainer, ['loss'])
trainer.run(train_loader, max_epochs=epochs)

### Exploring the tracked data with Aim

Load Aim extension for notebooks:

In [ ]:
%load_ext aim

Run `%aim up` to open Aim UI in the notebook:

In [ ]:
%aim up

### References
* [Convolutional Neural Networks for Classifying Fashion-MNIST Dataset using Ignite](https://github.com/pytorch/ignite/blob/master/examples/notebooks/FashionMNIST.ipynb)